<a href="https://colab.research.google.com/github/anydevil0812/bootcamp_project/blob/main/%EB%8C%84%EC%8A%A4_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re
import tensorflow as tf
import os
from tensorflow.keras import layers
from tensorflow.keras import activations
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from tqdm.notebook import tqdm, trange

In [ ]:
data_dir = '/content/drive/MyDrive/ITstudy/Project2'

In [ ]:
os.listdir(data_dir)

['dataset.zip',
 'dataset',
 '댄스_작사 모델링.ipynb의 사본',
 '댄스_모델1의 사본',
 '댄스_모델2 (크롤링)_______댄스곡 크롤링 오류 확인 요망의 사본',
 '발표자료최종본.ipynb',
 '댄스_프로젝트.ipynb']

In [ ]:
import argparse
args = {
    "seed": 1234
}
args = argparse.Namespace(**args)

print(args)

Namespace(seed=1234)


In [ ]:
args.corpus = os.path.join(data_dir, 'dataset.zip')

In [ ]:
args.corpus

'/content/drive/MyDrive/ITstudy/Project2/dataset.zip'

In [ ]:
# 댄스 데이터 불러오기

raw_corpus = []


with open("/content/drive/MyDrive/ITstudy/Project2/dataset/1990-2000_댄스.tsv", "r") as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 205
Examples:
 ['lyrics', '이제 모든걸 다시 시작해 내겐 아직도 시간이 있어 때론 상처가 좌절로 남아 돌이킬수 없는 후회도 하고 그러나 우리 잊어선 안돼 지금의 나는 내가 아닌걸 신문에 실려온 얘기들 헝클어진 우리들을 탓할순 없어 이제 모든걸 다시 시작해 이렇게 여기서 끝낼순 없어 내겐 아직도 시간이 있어 지금 이렇게 지금 멈출수는 없어 신문에 실려온 얘기들 헝클어진 우리들을 탓할순 없어 이제 모든걸 다시 시작해 이렇게 여기서 끝낼순 없어 내겐 아직도 시간이 있어 지금 이렇게 지금 나는 여기서 멈출순 없어 이제 모든걸 다시 시작해 나는 여기서 멈출순 없어 이제 모든걸 다시 시작해', '워 우리 모두 사랑하자 우리의 젊은날을 위하여 우리 모두 춤을 추자 가벼운 인스텝 속에 그대와 함께 춤을 아무렇게나 옷을 입고 거리를 걷다보면은 스쳐가는 사람들속에 시선도 받지만 나름대로의 연출속에 자신감도 갖고 있고 그것이 어떤 것이든 우리 맘대로 오늘을 얘기하자 워 우리 모두 사랑하자 우리의 젊은날을 위하여 우리 모두 춤을 추자 가벼운 인스텝 속에 그대와 함께 춤을 댄스뮤직을 좋아하고 자동차도 좋아하고 이번에 새로 나온 청바지도 좋지만 지금 당장 내겐 이런 걸 재밌있게 떠들면서 얘기할 나의 진실한 그런 사랑도 친구도 여기 없어 워 우리 모두 사랑하자 우리의 젊은 날을 위하여 우리 모두 춤을 추자 가벼운 인스텝 속에 그대와 함께 춤을 솔직하게 말을 해봐 괜찮으니까 사랑한다 말하는게 뭐 힘들어요 아무것도 꾸밈없는 진실한 얘기로 더 늦기전에 우리 모두 사랑하자 워 우리 모두 사랑하자 우리의 젊은날을 위하여 우리 모두 춤을 추자 가벼운 인스텝 속에 그대와 함께 우리 모두 사랑하자 우리의 젊은날을 위하여 우리 모두 춤을 추자 가벼운 인스텝 속에 그대와 함께', '끝없는 어둠 속을 천천히 걷고 있어 어디서 본 듯한 야릇한 느낌이야 거리는 비에 젖어 싸늘함 더 해지고 바람은 얼어버린 내 어깨를 스쳐가네 ZAM 예 ZAM 저 멀리 다가오는 두 그림자의 모

In [ ]:
Y = [0 for i in range(205)]

In [ ]:
len(Y)

205

In [ ]:
Y = np.array(Y)

In [ ]:
word_to_id_tmp = {'[PAD]': 0, '[UNK]': 1}
for idx, sentence in enumerate(raw_corpus):
    # 길이가 0이면 패스
    if len(sentence) == 0: continue
    # 문장의 끝이 :이면 패스
    if sentence[-1] == ":": continue 
    #10개 문장만 먼저 보겠습니다.
    if idx >9: break 
        
print(sentence) 

언제나 함께 했던 시간들을 접어 두고서 이제 너는 떠나가네 즐거웠던 시간만을 기억해 줄래 해맑은 네 큰눈에 반했던게 어제 같은데 이제 너는 떠나가네 우리에 따뜻했던 마음 기억해 줘 신비한 채로 가려져 있던 널 알기도 전에 우리가 헤어지는 일은 없을거라 난 믿었는데 너를 보내기 위한 춤을 추고 있어 짧은 시간만이 남았어 내 친구야 창밖을 봐 눈이 오잖아 모두 너를 위한 거야 느낄 수 있니 아침 해가 뜨기전에 널 보내 줄께 내겐 언제나 짜릿함으로 다가왔던 너에 따뜻한 미소 우린 언제나 다함께 나누었던 웃음과 눈물 신비롭던 너에 몸짓이 하나하나가 나의 기억속엔 언제나 매일매일 생각나겠지 어쩌다 너의길이 힘겨워 걷다가 지칠땐 주저 앉아 잠시 울어 언젠가 다시 만날 그날을 생각해 친구야 하루가 다 지나가고 잠들기전에 우리들을 기억해줘 다시 만날땐 너를 꼭 안아 주겠어. 신비한 채로 가려져 있던 널 알기도 전에 우리가 헤어지는 일은 없을거라 난 믿었는데 너를 보내기 위한 춤을 추고 있어 짧은 시간만이 남았어 내 친구야 창밖을 봐 눈이 오잖아 모두 너를 위한 거야 느낄 수 있니 아침 해가 뜨기전에 널 보내 줄께


In [ ]:
# corpus라는 폴더를 생성하여 문장이 없거나 ':'를 가진 문장은 넘기고 저장을 진행

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(sentence)
        
corpus[1] # 곡 하나에 대한 가사

'이제 모든걸 다시 시작해 내겐 아직도 시간이 있어 때론 상처가 좌절로 남아 돌이킬수 없는 후회도 하고 그러나 우리 잊어선 안돼 지금의 나는 내가 아닌걸 신문에 실려온 얘기들 헝클어진 우리들을 탓할순 없어 이제 모든걸 다시 시작해 이렇게 여기서 끝낼순 없어 내겐 아직도 시간이 있어 지금 이렇게 지금 멈출수는 없어 신문에 실려온 얘기들 헝클어진 우리들을 탓할순 없어 이제 모든걸 다시 시작해 이렇게 여기서 끝낼순 없어 내겐 아직도 시간이 있어 지금 이렇게 지금 나는 여기서 멈출순 없어 이제 모든걸 다시 시작해 나는 여기서 멈출순 없어 이제 모든걸 다시 시작해'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenizer와 pad_sequences를 사용한 문장 전처리
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=40000) # 빈도수대로 4만개의 단어만 토큰화(사전화)
tokenizer.fit_on_texts(corpus) # 토크나이저에 각 문장을 실제로 입력
tensor = tokenizer.texts_to_sequences(corpus) 

In [ ]:
tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=113) 

In [ ]:
len(tensor)

205

In [ ]:
len(tensor[0])

113

In [ ]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

In [ ]:
len(src_input[2])

112

In [ ]:
tgt_input

array([[   0,    0,    0, ...,    0,    0,    0],
       [ 117,   20,  531, ...,    0,    0,    0],
       [4673,   27, 1973, ...,   74,  280,   31],
       ...,
       [ 331,    3,   68, ...,  166,   33,  252],
       [1899,    6,   52, ...,  158,  740,  158],
       [  95,   95, 1187, ...,   44,   44,   44]], dtype=int32)

In [ ]:
len(src_input[2])

112

In [ ]:
src_input[8]

array([ 389,  357, 2925,   30,    6, 2907,   55,    7, 2908,  160,  174,
       2909,   98, 4792, 2910, 1998,   55,   85, 2911, 2912,    2,   98,
         99,  387,    6,  356, 2913,   49,  267, 1505,   30,  874,  484,
         92, 1506, 1024,   58,  601,  267, 1505,   30,  874,  484,   92,
       1506, 1024,   58,  601,   15,  123, 2919,   14,  672, 2920,    8,
        217, 2921,   36,  541, 1507, 2926,  132, 1025,    8, 2922,  447,
        159,    6, 2923,    6,  875,  308,    9,   13,    2,   15,   41,
       2914, 2915,  271,    6,  216, 2916,  163,   34,   41, 2917,  285,
         26,  873, 2918,    8,  601, 1211,   41,  267,  124,  876,    7,
        421, 4793, 2924,  748,   41,  132,  105, 2000,   58,    4,  749,
        877,    9], dtype=int32)

In [ ]:
len(tokenizer.index_word)

10213

In [ ]:
tokenizer.index_word[4661]

'lyrics'

In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : 내
2 : 난
3 : you
4 : 너를
5 : 널
6 : 너의
7 : 나를
8 : 날
9 : 수
10 : 그


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True, random_state=121)

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Embedding() - 단어를 밀집 벡터로 만드는 역할
#             - 인공 신경망 용어로는 임베딩 층(embedding layer)을 만드는 역할을 합니다. 
#             - Embedding()은 정수 인코딩이 된 단어들을 입력을 받아서 임베딩을 수행합니다.

from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = Embedding(vocab_size, embedding_size)
        self.rnn_1 = LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = LSTM(hidden_size, return_sequences=True)
        self.linear = Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

In [ ]:
vocab_size = tokenizer.num_words + 1
embedding_size = 700
hidden_size = 350
# drop_rate = 0.2
model = TextGenerator(vocab_size, embedding_size , hidden_size)

In [ ]:
from tensorflow.python.ops.nn_ops import dropout
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(X_train, y_train, epochs=200, batch_size=32)

Epoch 1/200
6/6 [==============================] - 4s 150ms/step - loss: 10.5748
Epoch 2/200
6/6 [==============================] - 1s 137ms/step - loss: 9.2813
Epoch 3/200
6/6 [==============================] - 1s 137ms/step - loss: 8.1030
Epoch 4/200
6/6 [==============================] - 1s 136ms/step - loss: 7.8848
Epoch 5/200
6/6 [==============================] - 1s 137ms/step - loss: 7.8081
Epoch 6/200
6/6 [==============================] - 1s 138ms/step - loss: 7.7966
Epoch 7/200
6/6 [==============================] - 1s 136ms/step - loss: 7.7901
Epoch 8/200
6/6 [==============================] - 1s 137ms/step - loss: 7.7671
Epoch 9/200
6/6 [==============================] - 1s 136ms/step - loss: 7.7747
Epoch 10/200
6/6 [==============================] - 1s 136ms/step - loss: 7.7655
Epoch 11/200
6/6 [==============================] - 1s 136ms/step - loss: 7.7597
Epoch 12/200
6/6 [==============================] - 1s 137ms/step - loss: 7.7523
Epoch 13/200
6/6 [==================

In [ ]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    # end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        # if predict_word.numpy()[0] == end_token: break
        # if test_tensor.shape[1] >= max_len: break


    generated = ""
        # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        if word_index == 0:
            continue
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 자연어 문장

In [ ]:
# 크롤링한 댄스 50곡으로 200번 학습한 결과, okt 사용, <start>, <end> 추가 

print('예시1.', generate_text(model, tokenizer, init_sentence='힘내', max_len=50))
print('예시2.', generate_text(model, tokenizer, init_sentence='슬픔', max_len=50))
print('예시3.', generate_text(model, tokenizer, init_sentence='비가 내려', max_len=50))
print('예시4.', generate_text(model, tokenizer, init_sentence='i love', max_len=50))

NameError: ignored

In [ ]:
# # 감성 분석 모델 정의
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(40000, 1, input_length=113),
#     tf.keras.layers.LSTM(units=30, return_sequences=True, input_shape=[100,2]), 
#     tf.keras.layers.LSTM(units=50),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dropout(rate=0.3),
#     tf.keras.layers.Dense(1, activation='softmax') 
# ])

# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 113, 1)            40000     
                                                                 
 lstm (LSTM)                 (None, 113, 30)           3840      
                                                                 
 lstm_1 (LSTM)               (None, 50)                16200     
                                                                 
 dense (Dense)               (None, 32)                1632      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 61,705
Trainable params: 61,705
Non-traina

In [ ]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
X_train.shape

(164, 113)

In [ ]:
y_train.shape

(164,)

In [ ]:
y_test.shape

(41,)

In [ ]:
# history = model.fit(X_train, y_train, epochs=10, batch_size=256, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=6, monitor='loss')])

Epoch 1/10
1/1 [==============================] - 9s 9s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 27ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 25ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 24ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 23ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 24ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [ ]:
# 글자수 세기
total_count = 0 

with open('/content/drive/MyDrive/ITstudy/Project2/dataset/1990-2000_댄스.tsv') as f:
  for i, line in enumerate(tqdm(f)):
    total_count += 1
print(total_count)

0it [00:04, ?it/s]

205


In [ ]:
# 빈도수 세기

import collections 
word_counter = collections.Counter()
word_counter.update(sentence.split())
word_counter

Counter({'1년을': 1,
         'baby': 2,
         'bye': 1,
         'come': 2,
         'e': 1,
         'good': 1,
         'goodbye': 1,
         'hey': 34,
         'love': 19,
         'me': 1,
         'n': 1,
         'o': 2,
         'on': 2,
         'w': 1,
         'y': 1,
         'you': 54,
         'you의': 1,
         '가다가': 1,
         '가자': 1,
         '가지': 1,
         '같지': 1,
         '걸려고': 1,
         '걸진': 1,
         '것': 1,
         '것만으로': 1,
         '괜찮아요': 1,
         '괜찮지': 1,
         '그': 1,
         '그게': 1,
         '그냥': 1,
         '그대': 3,
         '그대로': 2,
         '그대를': 1,
         '그러는거': 1,
         '그렇겠지': 1,
         '그렇지': 1,
         '기다렸는데': 1,
         '꾸미고': 1,
         '끌어': 1,
         '나': 4,
         '나는': 1,
         '나를': 1,
         '나만의': 1,
         '난': 2,
         '날려': 1,
         '남자': 4,
         '남자를': 1,
         '내': 4,
         '내가': 1,
         '내게로': 2,
         '내려': 1,
         '내맘이': 1,
         '너를': 4,
         '너만

In [ ]:
with open('/content/drive/MyDrive/ITstudy/Project2/dataset/1990-2000_댄스.tsv') as f:
  for line in tqdm(f, total=total_count):
      word_counter.update(list(line))

  0%|          | 0/205 [00:00<?, ?it/s]

In [ ]:
word_counter.items()

dict_items([('hey', 34), ('you', 54), ('love', 19), ('첫번째', 1), ('내', 1343), ('남자', 4), ('그대로', 2), ('떠나', 2), ('보내고', 2), ('good', 1), ('bye', 1), ('저기', 2), ('저', 196), ('내게로', 2), ('어서오세요', 2), ('baby', 2), ('매일', 2), ('난', 549), ('한', 781), ('사람에', 1), ('빠져', 1), ('상상으로', 1), ('빠져들곤', 1), ('하지', 1), ('그렇지', 1), ('그', 1579), ('사람을', 1), ('위해', 3), ('살아', 1), ('가지', 1), ('어디선가', 1), ('본', 46), ('적없는', 1), ('어느새', 1), ('사람이', 1), ('되어', 1), ('머릿속에', 1), ('빙글', 2), ('사랑인', 1), ('것', 327), ('같지', 1), ('모두가', 1), ('그렇겠지', 1), ('도무지', 1), ('알', 220), ('수', 608), ('없어', 5), ('살', 179), ('수가', 1), ('일년에', 1), ('입을', 1), ('옷', 12), ('몇', 20), ('일만에', 1), ('입고', 1), ('너를', 4), ('놓치기', 1), ('싫어하지', 1), ('혼자의', 1), ('시간이', 1), ('어딨겠어', 1), ('나는', 1), ('너와', 1), ('함께', 1), ('하고', 1), ('지내고', 1), ('사랑하고', 1), ('나만의', 1), ('시간', 1), ('이젠', 1), ('날려', 1), ('버려', 1), ('그러는거', 1), ('아니', 2), ('이러는거', 1), ('상상이나', 1), ('되니', 1), ('어처구니', 1), ('없이', 1), ('돌아가는', 1), ('사랑인줄', 1), ('알면서도', 1), ('그냥', 1),